<a href="https://colab.research.google.com/github/neuromatch/NeuroAI_Course/blob/main/tutorials/W1D5_Microcircuits/student/W1D5_Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/neuromatch/NeuroAI_Course/main/tutorials/W1D5_Microcircuits/student/W1D5_Tutorial1.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

# Bonus Material

**Week 1, Day 5: Microcircuits**

By Neuromatch Academy

**Content creators**: Noga Mudrik, Xaq Pitkow

**Content reviewers**: Yizhou Chen, RyeongKyung Yoon, Ruiyi Zhang, Lily Chamakura, Hlib Solodzhuk, Patrick Mineault, Alex Murphy

**Production editors**: Konstantine Tsafatinos, Ella Batty, Spiros Chavlis, Samuele Bolotta, Hlib Solodzhuk, Alex Murphy


___


# Tutorial Objectives

We will continue to explore the notion of sparsity, but this time in relation to the spatial domain. If you recall from Tutorial 1, there we were examining the temporal dimension primarily. Our exercises on spatial sparsity and spatial differentiation will guide you to view the notion of sparsity through a slightly different lens. We hope this alternate perspective can further reinforce how sparsity is a key component driving generalization and how this works in both the brain and in AI models.

The sections below contain bonus exercises for the following tutorials you have seen today.

* Section 1: Sparsity
* Section 2: Sparsity
* Section 3: Normalization


With all that said, let's get to work!


---
# Setup



In [ ]:
# @title Install and import feedback gadget

!pip install vibecheck datatops --quiet

from vibecheck import DatatopsContentReviewContainer
def content_review(notebook_section: str):
    return DatatopsContentReviewContainer(
        "",  # No text prompt
        notebook_section,
        {
            "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
            "name": "neuromatch_neuroai",
            "user_key": "wb2cxze8",
        },
    ).render()


feedback_prefix = "W1D5_T4"

In [ ]:
# @title Imports

#working with data
import numpy as np
import pandas as pd
from scipy.stats import kurtosis

#plotting
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import matplotlib.patheffects as path_effects

#interactive display
import ipywidgets as widgets
from ipywidgets import interact, IntSlider
from tqdm.notebook import tqdm as tqdm

#modeling
from sklearn.datasets import make_sparse_coded_signal, make_regression
from sklearn.decomposition import DictionaryLearning, PCA
from sklearn.linear_model import OrthogonalMatchingPursuit
import tensorflow as tf

#utils
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# @title Figure settings

logging.getLogger('matplotlib.font_manager').disabled = True
sns.set_context('talk')

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # perfrom high definition rendering for images and plots
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

---
# Bonus Section 1: Sparsity in space

In this section, we will focus on sparsity not from the perspective of the temporal domain (which we saw in the main tutorial) but rather from the spatial domain, how sparsity between units can be quantified and measured.

In [ ]:
# @title Bonus Video 1: Sparsity in space

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'bGH4NiD1Ye4'), ('Bilibili', 'BV1sT421a7MX')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_sparsity_in_space")

## Bonus Coding Excercise 1: Hard thresholding

Below, you can find the 1st frame, which we will focus on for the spatial differentiation (i.e., differentiation in space).

In [ ]:
# @title Visualization of the frame

frame = np.load('frame1.npy')
plot_images([frame])

So far, we have focused on applying ReLU, which can also be considered as a form of **'soft thresholding'**. Now, we will apply a different kind of thresholding called '**hard thresholding**'.

Here, you will have to write a function called **hard_thres** that receives 2 inputs:

1) 'frame' - 2d array as input of $p \times p$ floats.

2) 'theta' - threshold scalar values.

The function has to return a 2d array called **frame_HT** with the same dimensions as of **frame**, however, all values smaller than $\theta$ need to be set to 0, i.e.:

 $$
\text{frame-HT}_{[i,j]} =
\begin{cases}
frame_{[i,j]} & \text{if } frame_{[i,j]} \geq \theta \\
0 & \text{otherwise}
\end{cases}
$$

for every pixel [i,j] in the frame.

In [ ]:
def hard_thres(frame, theta):
    """
    Return a hard thresholded array of values based on the parameter value theta.

    Inputs:
    - frame (np.array): 2D signal.
    - theta (float, default = 0): threshold parameter.
    """
    ###################################################################
    ## Fill out the following then remove.
    raise NotImplementedError("Student exercise: complete hard thresholding function.")
    ###################################################################
    frame_HT = frame.copy()
    frame_HT[...] = 0
    return frame_HT

In [ ]:
#to_remove solution
def hard_thres(frame, theta):
    """
    Return a hard thresholded array of values based on the parameter value theta.

    Inputs:
    - frame (np.array): 2D signal.
    - theta (float, default = 0): threshold parameter.
    """
    frame_HT = frame.copy()
    frame_HT[frame_HT < theta] = 0
    return frame_HT

In [ ]:
# @title Test your function

frame_HT = hard_thres(frame, 150)
plot_images([frame, frame_HT])

Notice how many pixels became of the same **violet** color. Above, the violet color corresponds to 0. Hence, these pixels are the **thresholded** ones.

Here, we will define the variable **frame_HT** as the first frame after the hard threshold operator, using the above **hard_thres** function with a threshold of 80% of the frame values.

In [ ]:
# @title Observe 80% of thresholded signal

low_perc = np.percentile(frame, 80)
frame_HT = hard_thres(frame, low_perc)
plot_images([frame, frame_HT])

Notice that even more pixels become violet as we threshold the vast majority of the signal (80%).

We will now explore the differences before and after applying hard thresholding using the histogram. Generate a histogram for the `frame_HT`. It might take a while as the image is of high quality and it contains a lot of pixels.

In [ ]:
# @title Histogram comparison

with plt.xkcd():
    fig, axs = plt.subplots(1,2,figsize = (15,5), sharey = True)
    axs[0].hist(frame.flatten(), bins = 100);
    axs[1].hist(frame_HT.flatten(), bins = 100);

    #utils
    [ax.set_yscale('log') for ax in axs]
    [remove_edges(ax) for ax in axs]
    [add_labels(ax, ylabel = 'Count', xlabel = 'Value') for ax in axs]
    [ax.set_title(title) for title, ax in zip(['Before Thresholding', 'After Thresholding'], axs)]

Let us also take a look at the kurtosis to assess sparsity.

In [ ]:
# @title Kurtosis values comparison

plot_labeled_kurtosis(frame, frame_HT)

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_hard_thresholding")

---
# Bonus Section 2: Spatial differentiation

In this section, we will apply differentiation operations to spatial data. Spatial differentiation is common in identifying image patterns and extracting meaningful features.

In [ ]:
# @title Bonus Video 2: Spatial differentiation

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'IpMDGo7WYpM'), ('Bilibili', 'BV1nS411N7fK')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_spatial_differentiation")

## Bonus Coding Exercise 2: Spatial filtering

Let's apply spatial differentiation on the 1st frame to look at the result.

Below, apply 1-frame spatial differentiation on **frame**.

Define a variable **diff_x** that stores the x-axis differentiation and **diff_y** that stores the y-axis differentiation. Compare the results by presenting the differentiated image.

In [ ]:
###################################################################
## Fill out the following then remove
raise NotImplementedError("Student exercise: complete calcualtion of `diff_x` and `diff_y`.")
###################################################################
diff_x = np.diff(frame, axis = ...)
diff_y = np.diff(...)

In [ ]:
#to_remove solution

diff_x = np.diff(frame, axis = 1)
diff_y = np.diff(frame, axis = 0)

In [ ]:
# @title Observe the result
plot_spatial_diff(frame, diff_x, diff_y)

Now, as usual, let's look at the histogram and kurtosis.

In [ ]:
# @title Observe the histogram
plot_spatial_histogram(frame, diff_x, diff_y)

In [ ]:
# @title Observe the kurtosis values
plot_spatial_kurtosis(frame, diff_x, diff_y)

### Multi-Step Analysis in Spatial Differencing

Rather than focusing on differences between adjacent pixels, we'll employ more comprehensive filters for spatial differencing. Specifically, we will apply the differentiation using the `diff_box` method, which is similar to our approach in the temporal case. Define the `diff_box_values_y` variable for the specified windows for the first frame in both `x` and `y' directions.

In contrast to the temporal differencing, we aim for the `diff_box` to be symmetric in the spatial case. This is important because, in spatial analysis, we need to consider the context from both sides of a given point, as opposed to time-series data, where there is a clear directional flow from past to present.

![Comparison of filters.](https://github.com/neuromatch/NeuroAI_Course/blob/main/tutorials/W1D5_Microcircuits/static/filters.png?raw=true)

Define a **symmetric** box filter in the function below.

In [ ]:
def diff_box_spatial(data, window, pad_size = 4):
    """
    Implement & apply spatial filter on the signal.

    Inputs:
    - data (np.ndarray): input signal.
    - window (int): size of the window.
    - pad_size (int, default = 4): size of pad around data.
    """
    ###################################################################
    ## Fill out the following then remove
    raise NotImplementedError("Student exercise: add pad around to complete filter operation.")
    ###################################################################
    filter = np.concatenate([np.repeat(0, ...), np.repeat(-1, window), np.array([1]), np.repeat(-1, window), np.repeat(0, ...)]).astype(float)

    #normalize
    filter /= np.sum(filter**2)**0.5

    #make sure the filter sums to 0
    filter_plus_sum =  filter[filter > 0].sum()
    filter_min_sum = np.abs(filter[filter < 0]).sum()
    filter[filter > 0] *= filter_min_sum/filter_plus_sum

    #convolution of the signal with the filter
    diff_box = np.convolve(data, filter, mode='full')[:len(data)]
    diff_box[:window] = diff_box[window]
    return diff_box, filter

In [ ]:
#to_remove solution
def diff_box_spatial(data, window, pad_size = 4):
    """
    Implement & apply spatial filter on the signal.

    Inputs:
    - data (np.ndarray): input signal.
    - window (int): size of the window.
    - pad_size (int, default = 4): size of pad around data.
    """
    filter = np.concatenate([np.repeat(0, pad_size), np.repeat(-1, window), np.array([1]), np.repeat(-1, window), np.repeat(0, pad_size)]).astype(float)

    #normalize
    filter /= np.sum(filter**2)**0.5

    #make sure the filter sums to 0
    filter_plus_sum =  filter[filter > 0].sum()
    filter_min_sum = np.abs(filter[filter < 0]).sum()
    filter[filter > 0] *= filter_min_sum/filter_plus_sum

    #convolution of the signal with the filter
    diff_box = np.convolve(data, filter, mode='full')[:len(data)]
    diff_box[:window] = diff_box[window]
    return diff_box, filter

Let's observe the filter. How it differs from the temporal one?

In [ ]:
# @title Visualization of the filter
window = 10
diff_box_signal, filter = diff_box_spatial(sig, window)

with plt.xkcd():
    fig, ax = plt.subplots()
    plot_e1 = np.arange(len(filter))
    plot_e2 = np.arange(len(filter)) + 1
    plot_edge_mean = 0.5*(plot_e1 + plot_e2)
    plot_edge = lists2list( [[e1 , e2] for e1 , e2 in zip(plot_e1, plot_e2)])
    ax.scatter(plot_edge_mean, filter, color = 'purple')
    ax.plot(plot_edge, np.repeat(filter, 2), alpha = 0.3, color = 'purple')
    add_labels(ax,ylabel = 'Filter value', title = 'Box Filter', xlabel = 'Space (pixel)')

Let's apply the filter to the data. What do you observe?

In [ ]:
###################################################################
## Fill out the following then remove.
raise NotImplementedError("Student exercise: complete calcualtion of `diff_box_values_y`.")
###################################################################

num_winds = 10
windows = np.linspace(2,92,num_winds)
rows = frame.shape[0]
cols = frame.shape[1]

diff_box_values_x = [np.array([diff_box_spatial(frame[row], int(window))[0]  for row in range(rows)]) for window in windows]

diff_box_values_y = [np.array([diff_box_spatial(frame[:, ...], int(window))[0] for col in range(cols)]) for window in windows]

In [ ]:
#to_remove solution

num_winds = 10
windows = np.linspace(2,92,num_winds)
rows = frame.shape[0]
cols = frame.shape[1]

diff_box_values_x = [np.array([diff_box_spatial(frame[row], int(window))[0]  for row in range(rows)]) for window in windows]

diff_box_values_y = [np.array([diff_box_spatial(frame[:, col], int(window))[0] for col in range(cols)]) for window in windows]

In [ ]:
# @title Observe the results

visualize_images_diff_box(frame, diff_box_values_x, diff_box_values_y, num_winds)

In [ ]:
# @title Kurtosis values comparison

with plt.xkcd():
    fig, ax = plt.subplots(figsize = (17,7))
    tauskur = [kurtosis(np.abs(frame - diff_box_values_i).flatten()) for diff_box_values_i in diff_box_values_x]
    tauskur_y = [kurtosis(np.abs(frame.T - diff_box_values_i).flatten()) for diff_box_values_i in diff_box_values_y]

    df1 = pd.DataFrame([kurtosis(sig)] + tauskur, index = ['Signal']+ ['$window = {%d}$'%tau for tau in windows], columns = ['kurtosis x'])
    df2 = pd.DataFrame([kurtosis(sig)] + tauskur_y, index = ['Signal']+ ['$window = {%d}$'%tau for tau in windows], columns = ['kurtosis y'])
    dfs = pd.concat([df1,df2], axis = 1)
    dfs.plot.barh(ax = ax, alpha = 0.5, color =['purple', 'pink'])
    remove_edges(ax)
    plt.show()

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_spatial_filtering")

---
# Bonus Section 3: Benefits of using normalization - Efficient Coding

This bonus material explores some of the ideas that were introduced in Tutorial 2 on **Normalization**. 

Non-linearities are critical for computation, but they can also lose information. We propose you look at a very simple example of how normalization can help preserve information through a network. In the exercise below, complete `HardTanh` and functions `LeakyHardTanh` (observe that the inverse of the latter is already here for you).

`HardTanh` is the function $f(x)$ which is defined as following:

$$f(x) = \begin{cases}
1, & \text{if } x > 1\\
x, & \text{if } -1 \leq x \leq 1\\
-1, & \text{if } x < -1
\end{cases}$$

while `LeakyHardTanh` is $f(x) = \text{HardTanh}(x) + \text{leak-slope}* x$.

In [ ]:
#################################################
## TODO: Implement the normalization example equation ##
# Fill remove the following line of code once you have completed the exercise:
raise NotImplementedError("Student exercise: complete missing calculations in `HardTanh` and `LeakyHardTanh` functions.")
#################################################

def HardTanh(x):
  """
  Calculate `tanh` output for the given input data.

  Inputs:
  - x (np.ndarray): input data.

  Outputs:
  - output (np.ndarray): `tanh(x)`.
  """
  min_val = -1
  max_val = 1
  output = np.copy(x)
  output[output>...] = ...
  output[output<...] = ...
  return output

def LeakyHardTanh(x, leak_slope=0.03):
  """
  Calculate `tanh` output for the given input data with the leaky term.

  Inputs:
  - x (np.ndarray): input data.
  - leak_slope (float, default = 0.03): leaky term.

  Outputs:
  - output (np.ndarray): `tanh(x)`.
  """
  output = np.copy(x)
  output = HardTanh(output) + ...*...
  return output

def InverseLeakyHardTanh(y, leak_slope=0.03):
  """
  Calculate input into the `tanh` function with the leaky term for the given output.

  Inputs:
  - y (np.array): output of leaky tanh function.
  - leak_slope (float, default = 0.03): leaky term.

  Outputs:
  - output (np.array): input into leaky tanh function.
  """
  ycopy = np.copy(y)
  output = np.where(
      np.abs(ycopy) >= 1+leak_slope, \
      (ycopy - np.sign(ycopy))/leak_slope, \
      ycopy/(1+leak_slope)
  )
  return output

In [ ]:
#to_remove solution
def HardTanh(x):
  """
  Calculates `tanh` output for the given input data.

  Inputs:
  - x (np.ndarray): input data.

  Outputs:
  - output (np.ndarray): `tanh(x)`.
  """
  min_val = -1
  max_val = 1
  output = np.copy(x)
  output[output>max_val] = max_val
  output[output<min_val] = min_val
  return output

def LeakyHardTanh(x, leak_slope=0.03):
  """
  Calculate `tanh` output for the given input data with the leaky term.

  Inputs:
  - x (np.ndarray): input data.
  - leak_slope (float, default = 0.03): leaky term.

  Outputs:
  - output (np.ndarray): `tanh(x)`.
  """
  output = np.copy(x)
  output = HardTanh(output) + leak_slope*output
  return output

def InverseLeakyHardTanh(y, leak_slope=0.03):
  """
  Calculate input into the `tanh` function with the leaky term for the given output.

  Inputs:
  - y (np.array): output of leaky tanh function.
  - leak_slope (float, default = 0.03): leaky term.

  Outputs:
  - output (np.array): input into leaky tanh function.
  """
  ycopy = np.copy(y)
  output = np.where(
      np.abs(ycopy) >= 1+leak_slope, \
      (ycopy - np.sign(ycopy))/leak_slope, \
      ycopy/(1+leak_slope)
  )
  return output

In [ ]:
# @title Visualize the functions

# with plt.xkcd():
fig, ax = plt.subplots(1, 3, figsize=(12, 4))
plot_vals = np.arange(-4, 4, 0.01)
leak_slope = 0.03
for i in range(3):
  # Set the spines (axes lines) to intersect at the center
  ax[i].spines['left'].set_position('zero')
  ax[i].spines['bottom'].set_position('zero')
  ax[i].spines['right'].set_color('none')
  ax[i].spines['top'].set_color('none')
  ax[i].set_xlabel('x', loc='right', fontsize=20)
ax[0].plot(plot_vals, LeakyHardTanh(plot_vals, leak_slope), '-k')
ax[0].set_title('LeakyHardTanh(x)', fontsize=14)
ax[1].plot(plot_vals, InverseLeakyHardTanh(plot_vals, leak_slope), '-k')
ax[1].set_title('InverseLeakyHardTanh(x)', fontsize=14)
ax[2].plot(plot_vals, InverseLeakyHardTanh(LeakyHardTanh(plot_vals, leak_slope), leak_slope), '-k')
ax[2].set_title('InverseLeakyHardTanh( LeakyHardTanh(x) )', fontsize=14)
plt.tight_layout()
plt.show()

Now, let's define an $n$-dimensional vector $\mathbf{x}$. This is our target latent variable, and we would like to preserve the information about it. However, $\mathbf{x}$ is corrupted by a multiplicative scalar nuisance variable, g: $\mathbf{y}$=g$\mathbf{x}$.

Downstream computation will use $\mathbf{y}$ by passing it through an element-wise non-linearity $f$ (that saturates beyond a certain input range) and adding noise. By doing so, we lose information -- potentially a lot of information if $g$ is large and pushes the inputs into the saturating part of the non-linearity.

If we knew $g$, then we could remove it by division and reduce the problem. Although we don't know $g$, we can still use Normalization as an estimate of $g$, divide by that estimate, and invert the non-linearity to recover an approximation of the original $\mathbf{x}$. Here we use a `LeakyHardTanh`, which almost saturates but is technically invertible.

Let's see if Normalization helps. We will compute the correlation between x and the estimate $\hat{x}$ and compare this correlation with and without the usage of the Normalization function.

Our information ($\mathbf{X}$) is a collection of 10-dimensional vectors, having 400 samples in total. $\mathbf{X} \in \mathbb{R}^{400 \times 10}$, each of the components are drawn from $ \mathcal{N}(0, 1)$. For each component for each vector in $\mathbf{X}$, we have a nuisance scaling factor $s \in \mathbb{R}^{400}$, $s \sim Exp(0.2)$

In [ ]:
# @title Data

def normalize(x, sigma, p, g):
  """
  Inputs:
  - x(np.ndarray): Input array (n_samples * n_dim)
  - sigma(float): Smoothing factor
  - p(int): p-norm
  - g(int): scaling factor

  Outputs:
  - xnorm (np.ndarray): normalized values.
  """
  # Raise the absolute value of x to the power p
  xp = np.power(np.abs(x), p)
  # Sum the x over the dimensions (n_dim) axis
  xp_sum = np.sum(np.power(np.abs(x), p), axis=1)
  # Correct the dimensions of xp_sum, and taking the average reduces the dimensions
  # Making xp_sum a row vector of shape (1, n_dim)
  xp_sum = np.expand_dims(xp_sum, axis=1)
  # Raise the sum to the power 1/p and add the smoothing factor (sigma)
  denominator = sigma + np.power(xp_sum, 1/p)
  # Scale the input data with a factor of g
  numerator = x*g
  # Calculate normalized x
  xnorm = numerator/denominator
  return xnorm

# data
n_samples = 400 # number of samples
n_dim = 10 # dimensions of each sample
latent_std = 1 # width of latent distribution

# nuisance
nuisance_scale = 5 # distribution width for nuisance scaling factor

# normalization
smoothing_factor = 0.1 # normalization smoothness - sigma
norm_p = 2 # Lp norm
norm_scale = 1 # normalization scale

# noise
noise_std = 0.05 # added noise standard deviation

# Non-Linearity
leak_slope = 0.001 # slope after leaky saturation

# random nuisance scaling for each example vector
nuisance = np.random.exponential(nuisance_scale, size=(n_samples, 1))
x_sec31 = np.random.normal(loc=0.0, scale=latent_std, size=(n_samples, n_dim))
y_sec31 = x_sec31 * nuisance # input vectors scaled by random nuisance
ynorm_sec31 = normalize(y_sec31, smoothing_factor, norm_p, norm_scale) * norm_scale # normalized vectors
noise = np.random.normal(loc=0.0, scale=noise_std, size=(n_samples, n_dim))

# without normalization
transmit_noisy_x = LeakyHardTanh(y_sec31, leak_slope) + noise
estimate_x = InverseLeakyHardTanh(transmit_noisy_x, leak_slope)
# with normalization
transmitNormalized_noisy_x = LeakyHardTanh(ynorm_sec31, leak_slope) + noise
estimateNormalized_x = InverseLeakyHardTanh(transmitNormalized_noisy_x, leak_slope)

Let's take a look at one of the dimensions of $\mathbf{x}$ and visualize it after nuisance scaling as well as after normalization.

$$\mathbf{x}_{norm} = \frac{g \mathbf{x}}{\sigma + \sqrt[p]{\Sigma_{i = 1}^{N} |x_{i}|^{p}}}$$

In [ ]:
# @title Visualize input
with plt.xkcd():
    sns.kdeplot(ynorm_sec31[:, 0], color='r', label='$(s \mathbf{x})_{norm}$')
    sns.kdeplot(x_sec31[:, 0], color='k', label='$\mathbf{x}$')
    sns.kdeplot(y_sec31[:, 0], color='b', label='$s \mathbf{x}$')
    plt.xlabel('Information (x)')
    plt.legend()
    plt.show()

Now, let's transmit this observable information through a network. In this example, the network is an element-wise `LeakyHardTanh`. Additionally, the transmission is noisy with transmission noise $n \sim \mathcal{N}(0, 0.05)$.

Hence, the transmitted signal is `LeakyHardTanh`($s\mathbf{x}$) + $n$.

In [ ]:
# @title Visualize noisy transmitted signal
with plt.xkcd():
    plt.figure(figsize=(7.5, 7.5))
    sns.kdeplot(LeakyHardTanh(y_sec31, leak_slope)[:, 0], linestyle='--', color='b', label=r'LeakyHardTanh$(s \mathbf{x})$')
    sns.kdeplot(transmit_noisy_x[:, 0],color='b', label=r'LeakyHardTanh$(s \mathbf{x})$+noise')
    sns.kdeplot(LeakyHardTanh(ynorm_sec31, leak_slope)[:, 0], linestyle='--', color='r', label='LeakyHardTanh$(s \mathbf{x})_{norm}$')
    sns.kdeplot(transmitNormalized_noisy_x[:, 0], color='r', label='LeakyHardTanh$(s \mathbf{x})_{norm}$+noise')
    plt.xlabel('Transmitted Signal')
    plt.legend()
    plt.show()

Let's estimate the true information by calculating the inverse of the network (`InverseLeakyHardTanh`).

In [ ]:
# @title Visualize estimated information
with plt.xkcd():
    sns.kdeplot(estimateNormalized_x[:, 0], color='r', label='$\mathbf{\hat{x}}_{norm}$')
    sns.kdeplot(x_sec31[:, 0], color='k', label='$\mathbf{x}$')
    sns.kdeplot(estimate_x[:, 0], color='b', label='$\mathbf{\hat{x}}$')
    plt.xlabel('Estimated information (x)')
    plt.xlim(-50, 50)
    plt.legend()
    plt.show()

Let's quantify how well we can estimate the true information by calculating R-squared values.

In [ ]:
# @title Plot correlation between estimated information and true information

with plt.xkcd():
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))

    # Plot x vs. estimated x
    x_ = x_sec31.reshape((-1, 1)).squeeze(-1)
    y_ = estimate_x.reshape((-1, 1)).squeeze(-1)
    sns.regplot(x=x_, y=y_, ax=ax[0], fit_reg=False)
    ax[0].set_xlabel('x')
    ax[0].set_ylabel(r'$\hat{x}$')
    # Calculate R-squared and p-value
    result = scipy.stats.linregress(x_, y_)
    ax[0].set_title(r'$\hat{x} \enspace vs. \enspace x, \enspace R^{2} = $' + \
                    f'{(result.rvalue**2):.2f}')
    ax[0].set_ylabel('$\hat{x}$', loc='bottom', fontsize=20)
    ax[0].set_ylim((-5, 5))

    # Plot x vs. estimated normalized x
    x_ = x_sec31.reshape((-1, 1)).squeeze(-1)
    y_ = estimateNormalized_x.reshape((-1, 1)).squeeze(-1)
    sns.regplot(x=x_, y=y_, ax=ax[1], fit_reg=False)
    ax[1].set_xlabel('x')
    ax[1].set_ylabel(r'$\hat{x_{norm}}$')
    # ax[1].set_ylim((-1.05, 1.05))
    ax[1].set_ylim((-5, 5))
    # Calculate R-squared and p-value
    result = scipy.stats.linregress(x_, y_)
    ax[1].set_title(r'$\hat{x_{norm}} \enspace vs. \enspace x, \enspace R^{2} = $' + \
                    f'{(result.rvalue**2):.2f}')
    ax[1].set_ylabel('$\hat{x_{norm}}$', loc='bottom', fontsize=20)

    for i in range(2):
      # Set the spines (axes lines) to intersect at the center
      ax[i].spines['left'].set_position('zero')
      ax[i].spines['bottom'].set_position('zero')
      ax[i].spines['right'].set_color('none')
      ax[i].spines['top'].set_color('none')
      ax[i].set_xlabel('x', loc='right', fontsize=20)
      ax[i].set_xlim((-4, 4))

    plt.tight_layout()
    plt.show()

We see that normalization helped us preserve the information through transmission by preventing saturation (constraining the information within a limited dynamic range).

### Bonus Think 1.1

1. We control the dynamic range for normalization. Does there exist an optimum range?

Let's control the range by manipulating the scaling factor ($g$).

$$\mathbf{x}_{norm} = \frac{g \mathbf{x}}{\sigma + \sqrt[p]{\Sigma_{1}^{N} |x_{i}|^{p}}}$$

We will plot the improvement in the correlation versus the range that the normalization produces (via scaling factor $g$).

In [ ]:
# @title Effect of scaling normalization ($g$)
norm_scales = np.arange(0.01, 5, 0.01)
improvements = []

x_ = x_sec31.reshape((-1, 1)).squeeze(-1)
y_ = estimate_x.reshape((-1, 1)).squeeze(-1)
result = scipy.stats.linregress(x_, y_)
nonnorm_r2 = result.rvalue

for norm_scale in norm_scales:
  ynorm_ = normalize(y_sec31, smoothing_factor, norm_p, norm_scale) # normalized vectors
  transmitNormalized_noisy_x = LeakyHardTanh(ynorm_, leak_slope) + noise
  estimateNormalized_x = InverseLeakyHardTanh(transmitNormalized_noisy_x, leak_slope)
  x_ = x_sec31.reshape((-1, 1)).squeeze(-1)
  y_ = estimateNormalized_x.reshape((-1, 1)).squeeze(-1)
  result = scipy.stats.linregress(x_, y_)
  norm_r2 = result.rvalue
  improvement = norm_r2/nonnorm_r2
  improvements.append(improvement)

with plt.xkcd():
    plt.figure(figsize=(5,5))
    plt.plot(norm_scales, improvements, '.')
    plt.ylim((-0.05, 2.05))
    plt.xlabel('Normalization scaling factor ($g$)')
    plt.ylabel(r'Improvement')
    plt.title(r'Improvement = $\frac{R^{2}(x, \hat{x}_\mathrm{scaled norm})}{R^{2}(x, \hat{x})}$')
    ax = plt.gca()
    for line in ax.get_lines():
      line.set_path_effects([path_effects.Normal()])
    plt.show()

There's an optimal normalization range: while being too narrow - the noise dominates, and with too wide - the saturation destroys information.

### Bonus Think 1.2

1. Thinking deeper: here, we have used normalization only to preserve the information, essentially by avoiding most of the non-linearity. Do you think the computation can gain an advantage by using saturation? How?

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_efficient_coding")